<a href="https://colab.research.google.com/github/meti-94/OpenQA/blob/main/train_evaluation_0_0_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
!pip install transformers -q

     |████████████████████████████████| 3.1 MB 13.6 MB/s 
     |████████████████████████████████| 895 kB 38.4 MB/s 
     |████████████████████████████████| 596 kB 40.4 MB/s 
     |████████████████████████████████| 59 kB 6.2 MB/s 
     |████████████████████████████████| 3.3 MB 42.1 MB/s 


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!git clone https://github.com/meti-94/OpenQA.git
%cd OpenQA/src/

Cloning into 'OpenQA'...
remote: Enumerating objects: 138, done.
remote: Counting objects: 100% (138/138), done.
remote: Compressing objects: 100% (110/110), done.
remote: Total 138 (delta 41), reused 104 (delta 21), pack-reused 0
Receiving objects: 100% (138/138), 88.94 MiB | 9.45 MiB/s, done.
Resolving deltas: 100% (41/41), done.
/content/OpenQA/src


In [5]:
!python train.py

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): huggingface.co:443
DEBUG:urllib3.connectionpool:https://huggingface.co:443 "HEAD /bert-base-uncased/resolve/main/config.json HTTP/1.1" 200 0
DEBUG:filelock:Attempting to acquire lock 140225904123728 on /root/.cache/huggingface/transformers/3c61d016573b14f7f008c02c4e51a366c67ab274726fe2910691e2a761acf43e.37395cee442ab11005bcd270f3c34464dc1704b715b5d7d52b1a461abe3b9e4e.lock
DEBUG:filelock:Lock 140225904123728 acquired on /root/.cache/huggingface/transformers/3c61d016573b14f7f008c02c4e51a366c67ab274726fe2910691e2a761acf43e.37395cee442ab11005bcd270f3c34464dc1704b715b5d7d52b1a461abe3b9e4e.lock
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): huggingface.co:443
DEBUG:urllib3.connectionpool:https://huggingface.co:443 "GET /bert-base-uncased/resolve/main/config.json HTTP/1.1" 200 570
Downloading: 100% 570/570 [00:00<00:00, 508kB/s]
DEBUG:filelock:Attempting to release lock 140225904123728 on /root/.cache/huggingface

In [ ]:
!cp /content/OpenQA/models/node_edge_bert.pt /content/drive/MyDrive/data_freebase/

In [21]:
from train import *
# from graph import *
import pandas as pd
from utils import get_hit
'''
	A simple script to fill article table 
'''
if __name__=='__main__':
	device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
	bert = BertModel.from_pretrained("bert-base-uncased")
	tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
	node_edge_detector = NodeEdgeDetector(bert, tokenizer, dropout=torch.tensor(0.5))
	optimizer = AdamW
	kw = {'lr':0.0002, 'weight_decay':0.1}
	tl = TrainingLoop(node_edge_detector, optimizer, True, **kw)
	loss = mse_loss
	tl.load()

	# RKBG = ReverbKnowledgeBase()
	wordstoberttokens_array, berttokenstoids_array, input_token_ids_array, nodes_borders_array, edges_spans_array, node_array, edge_array = [], [], [], [], [], [], []
	questions_array = []
	freebase_df = pd.read_excel('/content/drive/MyDrive/data_freebase/test_useful_records.xlsx')
	reverb_df = pd.read_excel('/content/drive/MyDrive/data_freebase/test.xlsx')
	test_df = pd.DataFrame({'Question':freebase_df.Question.to_list()+reverb_df.Question.to_list()})
	# actual = test_df['Reverb_no'].to_list()
	system_results, candidates_array, actual_answer_array = [], [], []
	for index, row in tqdm(test_df.iterrows(), total=test_df.shape[0]):
		wordstoberttokens, berttokenstoids, input_token_ids, nodes_borders, edges_spans, node, edge = tl.readable_predict_article(
                                                device, _input=row['Question'], print_result=False)
		wordstoberttokens_array.append(wordstoberttokens)
		berttokenstoids_array.append(berttokenstoids)
		input_token_ids_array.append(input_token_ids)
		nodes_borders_array.append(nodes_borders)
		edges_spans_array.append(edges_spans)
    
		node = ' '.join(node); edge = ' '.join(edge)
		node = node.replace(' ##', ''); edge = edge.replace(' ##', '')

		node_array.append(node)
		edge_array.append(edge)
		questions_array.append(row['Question'].lower().split())
		# temp = RKBG.tfidf_query(node=node, edge=edge)
		# candidates_array.append(temp[:min(len(temp), 25)])
		# actual_answer_array.append(row['Reverb_no'])
	output_data = {
    # 'bert_tokenizer_output':wordstoberttokens_array,
    # 'bert_token_ids':berttokenstoids_array,
		'Question':test_df.Question.to_list(),
    'input_token_ids':input_token_ids_array,
    'nodes_borders':nodes_borders_array,
    'edges_spans':edges_spans_array,
    'node':node_array,
    'edges':edge_array,
    'question':questions_array,
    # 'candidates':candidates_array, 
    # 'actual_answer':actual_answer_array
  }

	pd.DataFrame(output_data).to_excel('/content/drive/MyDrive/data_freebase/article_step_by_step_output.xlsx')

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): huggingface.co:443
DEBUG:urllib3.connectionpool:https://huggingface.co:443 "HEAD /bert-base-uncased/resolve/main/config.json HTTP/1.1" 200 0
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): huggingface.co:443
DEBUG:urllib3.connectionpool:https://huggingface.co:443 "HEAD /bert-base-uncased/resolve/main/pytorch_model.bin HTTP/1.1" 302 0
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification mode

In [22]:
print(len(freebase_df.Question.to_list()), len(reverb_df.Question.to_list()))

21619 5003
